# DATA PROCESSING`

# Импорт необходимых модумей

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import locale
import re
import os
import logging

# Загрузка данных

In [4]:
# Настройка логирования для отслеживания выполнения кода
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Устанавливание рабочей директории
os.chdir("source/")

# Отображать все столбцы
pd.set_option('display.max_columns', None)

# Отображать все строки
pd.set_option('display.max_rows', None)


In [42]:
# Загрузка данных из CSV файлов
data_loan = pd.read_csv('ListPI1.csv')
data_app = pd.read_csv('ListP1.csv')
data_beh = pd.read_csv('ListB1.csv')


C:\Users\F_Jumaev\AppData\Local\Temp\ipykernel_16248\3632514673.py:2: DtypeWarning: Columns (17,18,25,26,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  data_loan = pd.read_csv('ListPI1.csv')
C:\Users\F_Jumaev\AppData\Local\Temp\ipykernel_16248\3632514673.py:4: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  data_beh = pd.read_csv('ListB1.csv')


# Преобразование названий столбцов в snake_case

In [43]:
# Функция для преобразования названий столбцов в snake_case
def to_snake_case(column_name):
    return column_name.lower().replace('.', '_').replace('#', 'number').replace(' ', '_')

logging.info("Преобразование названий столбцов в snake_case.")
data_loan.columns = [to_snake_case(col) for col in data_loan.columns]
data_app.columns = [to_snake_case(col) for col in data_app.columns]
data_beh.columns = [to_snake_case(col) for col in data_beh.columns]


2024-12-04 16:15:01,733 - INFO - Преобразование названий столбцов в snake_case.


# Переименование столбцов с русскими именами на английские

In [44]:
rename_columns = {
    'наличие_эсхата_онлайн': 'eskhata_online',
    'наличие_пластиковых_карт': 'plastic_cards',
    'наличие_депозита': 'deposit',
    'кумулятивная_просрочка': 'cumulative_delinquency',
    'рейтинг_бки': 'bki_rating',
    'количество_кредитов__в_бки_(заемщик)': 'bki_number_of_loans',
    'состояние': 'state',
    'причина_отказа': 'rejection_reason',
    'кол-во_пролонгации': 'number_of_extensions'
    
}
logging.info("Переименование столбцов с русскими именами на английские.")
data_loan.rename(columns=rename_columns, inplace=True)
data_app.rename(columns=rename_columns, inplace=True)
data_beh.rename(columns=rename_columns, inplace=True)


2024-12-04 16:15:01,750 - INFO - Переименование столбцов с русскими именами на английские.


# Подсчет количества дубликатов (не включая первую строку)

In [45]:
logging.info("Проверка дублирующихся строк по account_id")
display(data_loan.duplicated(subset='account_id').sum())
display(data_app.duplicated(subset='account_id').sum())
display(data_beh.duplicated(subset='account_id').sum())

logging.info("Проверка дублирующихся строк по customer_id")
display(data_loan.duplicated(subset='account_id').sum())
display(data_app.duplicated(subset='account_id').sum())
display(data_beh.duplicated(subset='account_id').sum())

logging.info("Подсчет количества дубликатов по application_id")
display(data_loan.duplicated(subset='application_id').sum())
display(data_app.duplicated(subset='application_id').sum())



2024-12-04 16:15:01,768 - INFO - Проверка дублирующихся строк по account_id


32801

0

0

2024-12-04 16:15:01,797 - INFO - Проверка дублирующихся строк по customer_id


...

0

0

2024-12-04 16:15:01,826 - INFO - Подсчет количества дубликатов по application_id


136

0

# Удаление дубликатов

In [46]:
# Удаление дубликатов по account_id
logging.info("Удаление дубликатов по account_id.")
data_loan.drop_duplicates(subset='account_id', keep='first', inplace=True)

# Удаление дубликатов по application_id
logging.info("Удаление дубликатов по application_id.")
data_loan.drop_duplicates(subset='application_id', keep='first', inplace=True)


# # Удаление дубликатов по customer_id
# logging.info("Удаление дубликатов по customer_id.")
# data_loan.drop_duplicates(subset='customer_id_loan', keep='first', inplace=True)

2024-12-04 16:15:01,929 - INFO - Удаление дубликатов по account_id.
2024-12-04 16:15:02,041 - INFO - Удаление дубликатов по application_id.


# Объединение данных по account_id

In [47]:
logging.info("Объединение данных по 'account_id'.")
res = data_loan.merge(data_app, on="account_id", how="left", suffixes=('_loan', '_app'))
res = res.merge(data_beh, on="account_id", how="left", suffixes=('', '_beh'))


2024-12-04 16:15:02,169 - INFO - Объединение данных по 'account_id'.


# Удаление дублириющих переменных

In [48]:
res = res.drop(['application_id_app', 'customer_id_app', 'customer_id'], axis=1)
res = res.rename(columns={
    'customer_id_loan': 'customer_id',
    'application_id_loan': 'application_id'
})


# Переименование столбцов для удобства

In [49]:
rename_columns = {
    'client_type_': 'client_type',
    'number_dependants':'dependants',
    'number_months_at_current_address':'months_at_current_address',
    'number_months_at_job':'months_at_job',
    'property_type/collateral_type': 'property_type',
    'instalment_amount/min_instalment_amount': 'instalment_amount',
    'amount_due_–_instalment': 'amount_due',
    'maximum_days_past_due': 'max_days_past_due',
    'maximum_days_past_due_lifetime': 'max_days_past_due_lifetime',
    'ftd-1': 'ftd_1',
    'ftd-2': 'ftd_2',
    'ftd-3': 'ftd_3',
    'ftd-4': 'ftd_4'
}

logging.info("Переименование столбцов")
res.rename(columns=rename_columns, inplace=True)


2024-12-04 16:15:04,038 - INFO - Переименование столбцов


In [50]:
res.head()

,customer_id,application_id,account_id,date_of_birth,gender,city_of_living,region_of_living,city_of_registration,region_of_registration,work_phone_number,mobile_phone_number,education,marital_status,dependants,number_children,months_at_current_address,employment_type,employment_sector,employment_segment,months_at_job,net_main_income,source_of_main_income,additional_income,source_of_additional_income,reported_expenses,months_with_bank,current_exposure,client_type,property_object,eskhata_online,plastic_cards,deposit,state,rejection_reason,branch_id,product_id,application_date,date_loan_granted,loan_amount,first_instalment_due_date,interest_rate,collateral_type,value_of_collateral,property_type,salary_payment_in_bank_account,loan_type,number_of_instalments,instalment_amount,run_date,date_account_opened,current_balance,date_last_payment,date_final_payment,due_date,payment_amount,account_status,number_of_payments_in_arrears,cumulative_delinquency,amount_due,principal_amount,interest_accrued,outstanding_balance,arrears_amount,current_days_past_due,max_days_past_due,max_days_past_due_lifetime,default_flag,number_of_extensions,bki_rating,bki_number_of_loans,ftd_1,ftd_2,ftd_3,ftd_4
0,2.512152e+10,764446/КР,3.561914e+10,1998-01-07 00:00:00,Женский,нохияи Фирдавси,Душанбе,Фирдавси,Душанбе,992000805085,NaN,Высшее,Не замужем,0,2,0,Имеет другой источник дохода,NaN,NaN,0,3121.36,Прочее,NaN,NaN,1272.0,36,0.0,0,Квартира,Да,Да,0.0,NaN,NaN,"ФИЛИАЛИ ЧСК ""БОНКИ ЭСХАТА"" ДАР Ш.ДУШАНБЕ",Карзхои гуногунмаксад,2021-10-19,2021-11-02,12300.0,2021-12-02,31.0,NaN,NaN,NaN,Нет,Многоцелевые кредиты_005_аннуитет,18.0,863.0,2023-06-30,2021-11-02,0.0,NaN,2023-04-20,NaN,NaN,Закрыт,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0
1,8.471401e+08,766801/КР,3.573316e+10,1969-05-16 00:00:00,Мужской,Истаравшан,Вилояти Сугд,Истаравшан,Вилояти Сугд,992985675558,NaN,Среднее,Женат,2,2,6,Собственный бизнес,Самозанятость,Услуги транспорта,276,3800.00,Прочее,NaN,NaN,1500.0,143,4691.17,1,Дом,Нет,Нет,0.0,NaN,NaN,"ФИЛИАЛИ ЧСК ""БОНКИ ЭСХАТА"" ДАР Ш.ИСТАРАВШАН",Карзхои гуногунмаксад,2021-10-25,2021-11-01,20000.0,2021-12-01,30.0,1) Поручитель;,1) 0;,1) Поручитель;,Нет,Многоцелевые кредиты_005_аннуитет,24.0,1118.0,2023-06-30,2021-11-01,0.0,2023-04-26,2023-10-26,2023-06-01,1118.0,Закрыт,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,5.0,0.0,0.0,0.0,0.0
2,6.286580e+09,766319/КР,3.573691e+10,1991-03-02 00:00:00,Мужской,Истаравшан,Вилояти Сугд,Истаравшан,Вилояти Сугд,992988621755,NaN,Среднее,Женат,3,2,9,Собственный бизнес,Самозанятость,Услуги транспорта,60,3500.00,Прочее,NaN,NaN,1000.0,61,4301.47,1,Дом,Нет,Нет,0.0,NaN,NaN,"ФИЛИАЛИ ЧСК ""БОНКИ ЭСХАТА"" ДАР Ш.ИСТАРАВШАН",Карзхои гуногунмаксад,2021-10-25,2021-11-05,10000.0,2021-12-06,30.0,NaN,NaN,NaN,Нет,Многоцелевые кредиты_005_аннуитет,24.0,559.0,2023-06-30,2021-11-05,0.0,2023-04-03,2023-10-26,2023-06-05,559.0,Закрыт,NaN,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,NaN,0.0,NaN,2.0,0.0,1.0,0.0,7.0
3,1.493983e+10,766446/КР,3.574159e+10,1966-10-23 00:00:00,Мужской,Фархор,Вилояти Хатлон,Пархар,Вилояти Хатлон,+992900078418; ; ;,NaN,Среднее,Женат,1,2,39,Имеет другой источник дохода,NaN,NaN,0,2400.00,Доход семьи,NaN,NaN,1000.0,60,1153.52,1,Дом,Нет,Нет,0.0,NaN,NaN,"ФИЛИАЛИ ЧСК ""БОНКИ ЭСХАТА"" ДАР Ш. КУЛОБ",Карзхои гуногунмаксад,2021-10-25,2021-11-03,3300.0,2021-12-03,31.0,NaN,NaN,NaN,Нет,Многоцелевые кредиты_005_аннуитет,12.0,323.0,2023-06-30,2021-11-03,0.0,2022-09-12,2022-10-26,NaN,NaN,Закрыт,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0
4,3.283014e+10,767392/КР,3.575473e+10,2001-06-07 00:00:00,Мужской,поселки Варзобского района,Нохияхои тобеи Чумхури,поселки Варзобского района,Нохияхои тобеи Чумхури,NaN,992888051515,Среднее,Холост,0,0,2,Собственный бизнес,Самозанятость,Услуги транспорта,36,5700.00,Предпринимательство,NaN,NaN,3001.0,20,4938.37,0,Дом,Да,Нет,0.0,NaN,NaN,"ФИЛИАЛИ ЧСК ""БОНКИ ЭСХАТА"" ДАР Ш.ДУШАНБЕ, Н.СИ...",Карзхои гуногунмаксад,2021-10-26,2021-11-03,5000.0,2021-12-03,31.0,NaN,NaN,

In [51]:
# Создание копии датафрейма для дальнейшей обработки
df = res.copy()

# Преобразование необходимых столбцов в числовые значения

In [52]:
numeric_columns = [
    'net_main_income', 'additional_income', 'months_with_bank',
    'reported_expenses', 'deposit', 'current_exposure'
]
missing_numeric = [col for col in numeric_columns if col not in df.columns]
if missing_numeric:
    logging.warning(f"Отсутствуют следующие числовые столбцы: {missing_numeric}")

existing_numeric_columns = [col for col in numeric_columns if col in df.columns]
logging.info("Преобразование числовых столбцов в числовые значения.")
df[existing_numeric_columns] = df[existing_numeric_columns].apply(pd.to_numeric, errors='coerce')

2024-12-04 16:15:04,234 - INFO - Преобразование числовых столбцов в числовые значения.


In [53]:
# Преобразование столбцов с датами в формат datetime
date_columns = [
    'application_date', 'date_of_birth', 'date_loan_granted', 'first_instalment_due_date',
    'date_last_payment', 'date_final_payment', 'due_date', 'date_account_opened'
]
logging.info("Преобразование столбцов с датами в формат datetime.")
for col in date_columns:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')


2024-12-04 16:15:04,795 - INFO - Преобразование столбцов с датами в формат datetime.


In [54]:

# Вычисление возраста
logging.info("Вычисление возраста.")
# Оптимизированное вычисление возраста
df['age'] = df['application_date'].dt.year - df['date_of_birth'].dt.year
# Корректировка возраста, если месяц или день рождения еще не наступили в году заявки
df.loc[
    (df['application_date'].dt.month < df['date_of_birth'].dt.month) |
    ((df['application_date'].dt.month == df['date_of_birth'].dt.month) &
     (df['application_date'].dt.day < df['date_of_birth'].dt.day)),
    'age'
] -= 1
# Удаление отрицательных возрастов, если такие есть
df['age'] = df['age'].where(df['age'] >= 0, np.nan)

2024-12-04 16:15:05,883 - INFO - Вычисление возраста.


In [55]:

# Создание столбца loan_month как год * 100 + месяц
logging.info("Создание столбца 'loan_month'.")
df['loan_month'] = df['application_date'].dt.year * 100 + df['application_date'].dt.month

# Признак совпадения города проживания и регистрации
logging.info("Создание признака совпадения города проживания и регистрации.")
df['city_of_living_eq_registration'] = (df['city_of_living'] == df['city_of_registration']).astype(int)

# Признак наличия залога
logging.info("Создание признака наличия залога.")
df['is_collateral'] = df['deposit'].notna().astype(int)

2024-12-04 16:15:06,096 - INFO - Создание столбца 'loan_month'.
2024-12-04 16:15:06,135 - INFO - Создание признака совпадения города проживания и регистрации.
2024-12-04 16:15:06,164 - INFO - Создание признака наличия залога.


In [56]:
# Создание временного датафрейма для анализа предыдущих заявок
logging.info("Создание временного DataFrame для анализа предыдущих заявок.")
temp_df = df.copy()

# Сортировка по customer_id и application_date для корректного сдвига
logging.info("Сортировка по 'customer_id' и 'application_date'.")
temp_df = temp_df.sort_values(['customer_id', 'application_date'])

# Добавление предыдущей даты заявки и предыдущей максимальной просрочки
logging.info("Добавление предыдущей даты заявки и предыдущей максимальной просрочки.")
temp_df['prev_application_date'] = temp_df.groupby('customer_id')['application_date'].shift(1)
temp_df['prev_max_days_past_due'] = temp_df.groupby('customer_id')['max_days_past_due_lifetime'].shift(1)


# Убираем записи без предыдущих заявок
logging.info("Удаление записей без предыдущих заявок.")
temp_df = temp_df.dropna(subset=['prev_application_date'])

2024-12-04 16:15:06,181 - INFO - Создание временного DataFrame для анализа предыдущих заявок.
2024-12-04 16:15:06,322 - INFO - Сортировка по 'customer_id' и 'application_date'.
2024-12-04 16:15:06,726 - INFO - Добавление предыдущей даты заявки и предыдущей максимальной просрочки.
2024-12-04 16:15:06,794 - INFO - Удаление записей без предыдущих заявок.


In [57]:
# Группировка по account_id для подсчета количества предыдущих заявок и максимальной просрочки
logging.info("Группировка по 'account_id' для подсчета количества предыдущих заявок и максимальной просрочки.")
acc_numb_before = temp_df.groupby('account_id').agg(
    cnt=('customer_id', 'count'),
    max_max_days_past_due=('prev_max_days_past_due', 'max')
).reset_index()

logging.info("Пример данных после группировки:")
print("\nacc_numb_before.head():")
print(acc_numb_before.head())


2024-12-04 16:15:06,954 - INFO - Группировка по 'account_id' для подсчета количества предыдущих заявок и максимальной просрочки.
2024-12-04 16:15:06,980 - INFO - Пример данных после группировки:



acc_numb_before.head():
     account_id  cnt  max_max_days_past_due
0  3.600220e+10    1                    4.0
1  3.600311e+10    1                   17.0
2  3.603127e+10    1                    0.0
3  3.605477e+10    1                    3.0
4  3.606057e+10    1                    0.0


In [58]:
# Добавление флага существующего клиента
logging.info("Добавление флага существующего клиента.")
acc_numb_before['existing_client'] = np.where(acc_numb_before['cnt'] > 1, 1, 0)

# Объединение с основным датафреймом
logging.info("Объединение 'acc_numb_before' с основным DataFrame.")
df = df.merge(
    acc_numb_before[['account_id', 'cnt', 'max_max_days_past_due']], 
    on='account_id', 
    how='left'
)

2024-12-04 16:15:07,316 - INFO - Добавление флага существующего клиента.
2024-12-04 16:15:07,320 - INFO - Объединение 'acc_numb_before' с основным DataFrame.


In [59]:
# Проверка наличия столбца 'cnt' после объединения
logging.info("Проверка наличия столбца 'cnt' после объединения.")
print("\nСтолбцы после объединения acc_numb_before:")
for column in df.columns.tolist():
    print(column)


if 'cnt' not in df.columns:
    logging.error("Столбец 'cnt' отсутствует в DataFrame после объединения. Проверьте корректность группировки и объединения.")
    raise KeyError("Столбец 'cnt' отсутствует в DataFrame после объединения.")
else:
    # Заполнение пропусков в previous_loans_count нулями
    logging.info("Заполнение пропусков в 'previous_loans_count' нулями.")
    df['previous_loans_count'] = df['cnt'].fillna(0).astype(int)
    
    # Создание бинарных признаков для просроченности с использованием .loc
    logging.info("Создание бинарных признаков для просроченности.")
    df.loc[:, 'gb_90ever'] = (df['max_days_past_due_lifetime'] >= 90).astype(int)
    df.loc[:, 'gb_cum_dlq_90'] = (df['cumulative_delinquency'] >= 90).astype(int)
    df.loc[:, 'gb_60ever'] = (df['max_days_past_due_lifetime'] >= 60).astype(int)
    
    # Удаление временных столбцов, если они больше не нужны
    logging.info("Удаление временных столбцов 'cnt' и 'max_max_days_past_due'.")
    df.drop(['cnt', 'max_max_days_past_due'], axis=1, inplace=True)

2024-12-04 16:15:08,018 - INFO - Проверка наличия столбца 'cnt' после объединения.
2024-12-04 16:15:08,020 - INFO - Заполнение пропусков в 'previous_loans_count' нулями.
2024-12-04 16:15:08,027 - INFO - Создание бинарных признаков для просроченности.
2024-12-04 16:15:08,034 - INFO - Удаление временных столбцов 'cnt' и 'max_max_days_past_due'.



Столбцы после объединения acc_numb_before:
customer_id
application_id
account_id
date_of_birth
gender
city_of_living
region_of_living
city_of_registration
region_of_registration
work_phone_number
mobile_phone_number
education
marital_status
dependants
number_children
months_at_current_address
employment_type
employment_sector
employment_segment
months_at_job
net_main_income
source_of_main_income
additional_income
source_of_additional_income
reported_expenses
months_with_bank
current_exposure
client_type
property_object
eskhata_online
plastic_cards
deposit
state
rejection_reason
branch_id
product_id
application_date
date_loan_granted
loan_amount
first_instalment_due_date
interest_rate
collateral_type
value_of_collateral
property_type
salary_payment_in_bank_account
loan_type
number_of_instalments
instalment_amount
run_date
date_account_opened
current_balance
date_last_payment
date_final_payment
due_date
payment_amount
account_status
number_of_payments_in_arrears
cumulative_delinquency
a

# Сводные статистики

In [60]:
logging.info("Сводные статистики по 'net_main_income'.")
print("\nСводные статистики по net_main_income:")
print(df['net_main_income'].describe())

logging.info("Сводные статистики по 'age'.")
print("\nСводные статистики по age:")
print(df['age'].describe())

logging.info("Сводные статистики по 'marital_status'.")
print("\nСводные статистики по marital_status:")
print(df['marital_status'].describe())

2024-12-04 16:15:14,107 - INFO - Сводные статистики по 'net_main_income'.
2024-12-04 16:15:14,128 - INFO - Сводные статистики по 'age'.
2024-12-04 16:15:14,141 - INFO - Сводные статистики по 'marital_status'.



Сводные статистики по net_main_income:
count    2.356630e+05
mean     5.173426e+03
std      4.147526e+05
min      1.000000e-02
25%      1.950000e+03
50%      2.787000e+03
75%      4.000000e+03
max      9.004431e+07
Name: net_main_income, dtype: float64

Сводные статистики по age:
count    235696.000000
mean         38.833540
std          12.371292
min           0.000000
25%          29.000000
50%          37.000000
75%          48.000000
max          94.000000
Name: age, dtype: float64

Сводные статистики по marital_status:
count     235424
unique         8
top        Женат
freq       99528
Name: marital_status, dtype: object


# Преобразование строковых переменных в категориальные

In [61]:
logging.info("Преобразование строковых переменных в категориальные.")
object_columns = df.select_dtypes(include=['object']).columns
df[object_columns] = df[object_columns].astype('category')

2024-12-04 16:15:26,203 - INFO - Преобразование строковых переменных в категориальные.


# Структура датафрейма после преобразований

In [88]:
logging.info("Просмотр структуры датафрейма после преобразований.")
print(df.info())

2024-12-04 16:39:32,164 - INFO - Просмотр структуры датафрейма после преобразований.


<class 'pandas.core.frame.DataFrame'>
Int64Index: 235699 entries, 0 to 235698
Data columns (total 82 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   customer_id                     235699 non-null  float64       
 1   application_id                  235699 non-null  category      
 2   account_id                      235698 non-null  float64       
 3   date_of_birth                   235698 non-null  datetime64[ns]
 4   gender                          235699 non-null  category      
 5   city_of_living                  235699 non-null  category      
 6   region_of_living                235699 non-null  category      
 7   city_of_registration            235699 non-null  category      
 8   region_of_registration          235699 non-null  category      
 9   work_phone_number               225416 non-null  category      
 10  mobile_phone_number             14543 non-null   categor

# Выбор необходимых столбцов

In [84]:
include_vars = [
    "customer_id",
    "account_id",
    "loan_amount",
    "is_collateral",
    "salary_payment_in_bank_account",
    "age",
    "gender",
    "region_of_living",
    "region_of_registration",
    "city_of_living_eq_registration",
    "education",
    "marital_status",
    "dependants",
    "months_at_current_address",
    "employment_type",
    "employment_segment",
    "months_at_job",
    "net_main_income",
    "source_of_main_income",
    "additional_income",
    "reported_expenses",
    "months_with_bank",
    "client_type",
    "property_object",
    "eskhata_online",
    "plastic_cards",
    "deposit",
    "gb_90ever",
    "gb_cum_dlq_90",
    "gb_60ever",
    "bki_rating",
    "bki_number_of_loans",
    "loan_month",
    "previous_loans_count",
    "max_days_past_due",
    
    
    # "collateral_type",
    # "date_of_birth",
    # "city_of_living",
    # "city_of_registration",
    # "employment_sector",
    # "source_of_additional_income",
    # "current_exposure",
    # "branch_id",
    # "product_id",
    # "application_id",
    # "work_phone_number",
    # "mobile_phone_number",
    # "number_children",
    # "state",
    # "rejection_reason",
    # "application_date",
    # "date_loan_granted",
    # "first_instalment_due_date",
    # "interest_rate",
    # "value_of_collateral",
    # "property_type",
    # "loan_type",
    # "number_of_instalments",
    # "instalment_amount",
    # "run_date",
    # "date_account_opened",
    # "current_balance",
    # "date_last_payment",
    # "date_final_payment",
    # "due_date",
    # "payment_amount",
    # "account_status",
    # "number_of_payments_in_arrears",
    # "cumulative_delinquency",
    # "amount_due",
    # "principal_amount",
    # "interest_accrued",
    # "outstanding_balance",
    # "arrears_amount",
    # "current_days_past_due",
    # "max_days_past_due_lifetime",
    # "default_flag",
    # "number_of_extensions",
    # "ftd_1",
    # "ftd_2",
    # "ftd_3",
    # "ftd_4"
]


In [85]:
# Проверка наличия всех столбцов
missing_columns = [col for col in include_vars if col not in df.columns]
if missing_columns:
    logging.warning(f"Отсутствуют следующие столбцы: {missing_columns}")

In [86]:
# Создание нового датафрейма с выбранными столбцами (только существующие)
existing_include_vars = [col for col in include_vars if col in df.columns]
df_selected = df[existing_include_vars].copy()

# Группировка по loan_month и вычисление показателей просроченности

In [66]:
logging.info("Группировка по 'loan_month' и вычисление показателей просроченности.")
rep_gb_90ever_month = df_selected.groupby('loan_month').agg(
    count=('gb_90ever', 'count'),
    bad_num=('gb_90ever', 'sum'),
    bad_rate=('gb_90ever', 'mean')
).reset_index()

rep_gb_cum_dlq_90_month = df_selected.groupby('loan_month').agg(
    count=('gb_cum_dlq_90', 'count'),
    bad_num=('gb_cum_dlq_90', 'sum'),
    bad_rate=('gb_cum_dlq_90', 'mean')
).reset_index()

rep_gb_60ever_month = df_selected.groupby('loan_month').agg(
    count=('gb_60ever', 'count'),
    bad_num=('gb_60ever', 'sum'),
    bad_rate=('gb_60ever', 'mean')
).reset_index()


2024-12-04 16:15:29,628 - INFO - Группировка по 'loan_month' и вычисление показателей просроченности.


# Создание выборок по типу занятости

In [67]:
logging.info("Создание выборок по типу занятости.")
sample_empl = df_selected[
    (df_selected['employment_type'] == "Работает в организации") &
    (df_selected['employment_segment'].isin([
        "Мед. работник",
        "Работник в сфере образования",
        "Работник госструктур",
        "Работник НПО (Ташкилоти Чамъияти)",
        "Работник производства",
        "Работник сельского хозяйство",
        "Работник частной организации",
        "Строитель",
        "Экономист"
    ]))
].copy()

sample_bus = df_selected[
    (df_selected['employment_type'] == "Собственный бизнес") &
    (df_selected['employment_segment'].isin([
        "Агро",
        "Производство",
        "Торговля",
        "Услуги",
        "Услуги Мастера",
        "Услуги транспорта"
    ]))
].copy()

sample_other = df_selected[
    (df_selected['employment_type'] == "Имеет другой источник дохода") &
    (df_selected['employment_segment'].isna())
].copy()

2024-12-04 16:15:29,697 - INFO - Создание выборок по типу занятости.


In [68]:
logging.info("Вывод размеров выборок.")
print(f"\nРазмер выборки для работников: {sample_empl.shape[0]}")
print(f"Размер выборки для бизнесменов: {sample_bus.shape[0]}")
print(f"Размер выборки для других источников дохода: {sample_other.shape[0]}")

2024-12-04 16:15:29,991 - INFO - Вывод размеров выборок.



Размер выборки для работников: 51547
Размер выборки для бизнесменов: 79305
Размер выборки для других источников дохода: 85248


# Создание выборки для анализа

In [69]:
# Пример выбора выборки для анализа
sample = sample_empl.copy()
# sample = sample_bus.copy()
# sample = sample_other.copy()

logging.info("Структура выбранной выборки:")
print(sample.info())

2024-12-04 16:15:30,471 - INFO - Структура выбранной выборки:



Структура выбранной выборки:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 51547 entries, 8 to 235698
Data columns (total 33 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   customer_id                     51547 non-null  float64 
 1   account_id                      51546 non-null  float64 
 2   loan_amount                     51546 non-null  float64 
 3   is_collateral                   51547 non-null  int32   
 4   salary_payment_in_bank_account  51546 non-null  category
 5   age                             51545 non-null  float64 
 6   gender                          51547 non-null  category
 7   region_of_living                51547 non-null  category
 8   region_of_registration          51547 non-null  category
 9   city_of_living_eq_registration  51547 non-null  int32   
 10  education                       48721 non-null  category
 11  marital_status                  51463 non-null  c

# Определение целевой переменной

In [70]:
# Создание переменной gb
logging.info("Создание целевой переменной 'gb'.")
sample.loc[:, 'gb'] = sample['gb_60ever']\

# sample.loc[:, 'gb'] = sample['gb_cum_dlq_90']

2024-12-04 16:15:30,890 - INFO - Создание целевой переменной 'gb'.


# Семплирование
## Создание выборок для Train и Test

In [71]:
# Выборка для обучения модели
dev = sample[(sample['loan_month'] >= 202110) & (sample['loan_month'] < 202210)].drop(columns=['loan_month']).copy()

# Выборка для тестирования модели на временную устойчивость
oot = sample[(sample['loan_month'] >= 202210) & (sample['loan_month'] < 202303)].copy()

# Выборка для кросс-валидации внутри обучающей выборки
oot2 = sample[(sample['loan_month'] >= 202207) & (sample['loan_month'] < 202210)].copy()

# Выборка для финального обучения модели
dev_final = sample[(sample['loan_month'] >= 202110) & (sample['loan_month'] < 202302)].drop(columns=['loan_month']).copy()

In [72]:
# Изучение распределений
logging.info("Изучение распределений в выборках.")
print("\nРаспределение в Development Sample:")
print(dev[['gb']].agg(['count', 'sum']))
print(dev[['gb']].agg(['count', 'sum']) / len(dev))

print("\nРаспределение в Out-of-time Sample:")
print(oot[['gb']].agg(['count', 'sum']))
print(oot[['gb']].agg(['count', 'sum']) / len(oot))

print("\nРаспределение в Out-of-time 2 Sample:")
print(oot2[['gb']].agg(['count', 'sum']))
print(oot2[['gb']].agg(['count', 'sum']) / len(oot2))

print("\nРаспределение в Development Final Sample:")
print(dev_final[['gb']].agg(['count', 'sum']))
print(dev_final[['gb']].agg(['count', 'sum']) / len(dev_final))

2024-12-04 16:15:31,902 - INFO - Изучение распределений в выборках.



Распределение в Development Sample:
          gb
count  27834
sum      599
            gb
count  1.00000
sum    0.02152

Распределение в Out-of-time Sample:
          gb
count  12429
sum       80
             gb
count  1.000000
sum    0.006437

Распределение в Out-of-time 2 Sample:
         gb
count  9355
sum     139
             gb
count  1.000000
sum    0.014858

Распределение в Development Final Sample:
          gb
count  37636
sum      669
             gb
count  1.000000
sum    0.017776


In [73]:
# Вывод количества строк в каждой выборке
logging.info("Вывод количества строк в каждой выборке.")
print(f"\nКоличество строк в Development Sample: {dev.shape[0]}")
print(f"Количество строк в Out-of-time Sample: {oot.shape[0]}")
print(f"Количество строк в Out-of-time 2 Sample: {oot2.shape[0]}")
print(f"Количество строк в Development Final Sample: {dev_final.shape[0]}")


2024-12-04 16:15:32,405 - INFO - Вывод количества строк в каждой выборке.



Количество строк в Development Sample: 27834
Количество строк в Out-of-time Sample: 12429
Количество строк в Out-of-time 2 Sample: 9355
Количество строк в Development Final Sample: 37636


# MACHINE LEARNING

In [90]:
import pandas as pd
import numpy as np
import scorecardpy as sc
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import json
import statsmodels.api as sm
import joblib



# Расчет Information Value (IV)

In [91]:
logging.info("Расчет Information Value (IV) для всех переменных в dev_final.")
iv_values_final = sc.iv(dt=dev_final, y='gb')


2024-12-04 16:48:02,223 - INFO - Расчет Information Value (IV) для всех переменных в dev_final.


In [92]:
# Просмотр структуры и первых строк DataFrame iv_values_final
print("Структура iv_values_final:")
print(iv_values_final.columns)

print("\nПервые 5 строк iv_values_final:")
print(iv_values_final.head())


Структура iv_values_final:
Index(['variable', 'info_value'], dtype='object')

Первые 5 строк iv_values_final:
             variable  info_value
15          gb_60ever   17.210712
4       gb_cum_dlq_90    7.762180
31          gb_90ever    6.407824
16  additional_income    2.209887
9         loan_amount    1.865620


# Отбор переменных на основе IV

In [93]:
logging.info("Отбор переменных на основе IV >= 0.01.")
selected_vars_iv = iv_values_final[
    iv_values_final['info_value'] >= 0.01
]['variable'].tolist()

logging.info(f"Переменные с info_value >= 0.01: {selected_vars_iv}")


2024-12-04 16:49:00,476 - INFO - Отбор переменных на основе IV >= 0.01.
2024-12-04 16:49:00,480 - INFO - Переменные с info_value >= 0.01: ['gb_60ever', 'gb_cum_dlq_90', 'gb_90ever', 'additional_income', 'loan_amount', 'reported_expenses', 'net_main_income', 'months_at_job', 'deposit', 'bki_number_of_loans', 'months_with_bank', 'age', 'employment_segment', 'marital_status', 'months_at_current_address', 'bki_rating', 'customer_id', 'salary_payment_in_bank_account', 'region_of_living', 'dependants', 'region_of_registration', 'education', 'eskhata_online', 'gender', 'property_object', 'city_of_living_eq_registration', 'source_of_main_income', 'plastic_cards', 'previous_loans_count']


# Вычисление доли пропусков для отобранных переменных

In [97]:
logging.info("Расчет доли пропусков для отобранных переменных.")
# Рассчитываем долю пропусков для отобранных переменных
missing_rates = dev_final[selected_vars_iv].isnull().mean().reset_index()
missing_rates.columns = ['variable', 'missing']


2024-12-04 16:51:15,466 - INFO - Расчет доли пропусков для отобранных переменных.


In [101]:
# Просмотр первых строк
logging.info("Структура missing_rates:")
display(missing_rates.columns)

logging.info("Первые 5 строк missing_rates:")
display(missing_rates.head())


2024-12-04 16:52:06,818 - INFO - Структура missing_rates:


Index(['variable', 'missing'], dtype='object')

2024-12-04 16:52:06,823 - INFO - Первые 5 строк missing_rates:


,variable,missing
0,gb_60ever,0.000000
1,gb_cum_dlq_90,0.000000
2,gb_90ever,0.000000
3,additional_income,0.545382
4,loan_amount,0.000000


# Объединение IV и missing_rates

In [100]:
# Объединяем IV и долю пропусков
iv_with_missing = iv_values_final.merge(missing_rates, on='variable')

# Просмотр первых строк
logging.info("Структура iv_with_missing:")
display(iv_with_missing.columns)

logging.info("Первые 5 строк iv_with_missing:")
display(iv_with_missing.head())


2024-12-04 16:52:01,805 - INFO - Структура iv_with_missing:


Index(['variable', 'info_value', 'missing'], dtype='object')

2024-12-04 16:52:01,811 - INFO - Первые 5 строк iv_with_missing:


,variable,info_value,missing
0,gb_60ever,17.210712,0.000000
1,gb_cum_dlq_90,7.762180,0.000000
2,gb_90ever,6.407824,0.000000
3,additional_income,2.209887,0.545382
4,loan_amount,1.865620,0.000000


# Фильтрация переменных по IV и пропускам

In [104]:
logging.info("Фильтрация переменных по info_value >= 0.01 и missing <= 0.95.")
# Отбор переменных с info_value >= 0.01 и пропусками <= 0.95
filtered_iv = iv_with_missing[
    (iv_with_missing['info_value'] >= 0.01) &
    (iv_with_missing['missing'] <= 0.95)
]

# Извлечение списка переменных
selected_vars_final = filtered_iv['variable'].tolist()

# Исключение определенных переменных
vars_to_remove = ["customer_id", "account_id", "gb_90ever", "gb_60ever", "gb_cum_dlq_90", "current_exposure"]
selected_vars_final = [var for var in selected_vars_final if var not in vars_to_remove]

logging.info(f"Отобранные переменные после фильтрации по IV и пропускам: {selected_vars_final}")


2024-12-04 16:52:46,894 - INFO - Фильтрация переменных по info_value >= 0.01 и missing <= 0.95.
2024-12-04 16:52:46,899 - INFO - Отобранные переменные после фильтрации по IV и пропускам: ['additional_income', 'loan_amount', 'reported_expenses', 'net_main_income', 'months_at_job', 'deposit', 'bki_number_of_loans', 'months_with_bank', 'age', 'employment_segment', 'marital_status', 'months_at_current_address', 'bki_rating', 'salary_payment_in_bank_account', 'region_of_living', 'dependants', 'region_of_registration', 'education', 'eskhata_online', 'gender', 'property_object', 'city_of_living_eq_registration', 'source_of_main_income', 'plastic_cards', 'previous_loans_count']


# Создание dev_final_sel

In [105]:
logging.info("Создание dev_final_sel с отобранными переменными.")
dev_final_sel = dev_final[selected_vars_final + ['gb']].copy()

logging.info(f"Количество строк в dev_final_sel: {dev_final_sel.shape[0]}")
logging.info(f"Количество столбцов в dev_final_sel: {dev_final_sel.shape[1]}")


2024-12-04 16:53:10,951 - INFO - Создание dev_final_sel с отобранными переменными.
2024-12-04 16:53:10,958 - INFO - Количество строк в dev_final_sel: 37636
2024-12-04 16:53:10,959 - INFO - Количество столбцов в dev_final_sel: 26


# Определение правил биннинга

In [107]:
logging.info("Определение правил биннинга для переменных.")

breaks = {
    'education': [
        "Высшее", "Ученая степень", "2 и более высших",
        "Среднее", "Начальное",
        "Среднее специальное",
        "Неоконченное высшее"
    ],
    'region_of_living': [
        "Вилояти Сугд", 
        "Вилояти Хатлон", "ВМКБ",
        "Душанбе", 
        "Нохияхои тобеи Чумхури"
    ],
    'marital_status': [
        "Женат", "Сожитель",
        "Замужем", "Вдова", "Разведена",
        "Холост", 
        "Не замужем", "Разведен"
    ],
    'employment_segment': [
        "Мед. работник", "Работник в сфере образования", "Экономист", "Работник НПО (Ташкилоти Чамъияти)",
        "Работник госструктур", "Строитель",
        "Работник производства", "Работник сельского хозяйство",
        "Работник частной организации"
    ],
    'source_of_main_income': [
        "Доход семьи",
        "Зарплата по основному месту работы",
        "Предпринимательство",
        "Пенсия", "Алименты", "Прочее", "missing"
    ],
    'months_at_job': [31, 61, 145, 277],
    'months_with_bank': [10, 25, 79],
    'net_main_income': ["missing", "1600", 3400]
}


2024-12-04 16:53:29,487 - INFO - Определение правил биннинга для переменных.
